In [ ]:
from enum import Enum
from queue import Queue, LifoQueue
from typing import TypeVar, List, Iterable, Callable, Collection, Sequence

X = TypeVar('X')

In [ ]:

class Operation(Enum):
    DELETE = 0
    CHANGE = 1
    INSERT = 2
    NO_CHANGE = 3


class EditTableCell:

    def __init__(self, weight: int, parent: 'EditTableCell' = None) -> None:
        super().__init__()
        self.weight = weight
        self.parent = parent
        self.comparator_weight = weight
        self.operation: Operation = Operation.NO_CHANGE

    def edit_sequence(self, queue = None):
        if queue is None:
            queue = LifoQueue()
        queue.put(self.operation)
        if self.parent is not None:
            return self.parent.edit_sequence(queue)
        return queue

def edit_distance(x: Sequence[X], y: Sequence[X], delta: Callable[[X, X], int] = lambda a, b: 0 if a == b else 1):
    edit_table = [[EditTableCell(0) for _ in range(len(y) + 1)] for _ in range(len(x) + 1)]
    for i in range(1, len(x) + 1):
        edit_table[i][0].parent = edit_table[i - 1][0]
        edit_table[i][0].weight = i
        edit_table[i][0].operation = Operation.DELETE
    for i in range(1, len(y) + 1):
        edit_table[0][i].parent = edit_table[0][i - 1]
        edit_table[0][i].weight = i
        edit_table[0][i].operation = Operation.INSERT
    for i in range(len(x)):
        k = i + 1
        for j in range(len(y)):
            l = j + 1
            edit_table[k - 1][l].comparator_weight = edit_table[k - 1][l].weight + 1
            edit_table[k][l - 1].comparator_weight = edit_table[k][l - 1].weight + 1
            edit_table[k - 1][l - 1].comparator_weight = edit_table[k - 1][l - 1].weight + delta(x[i], y[j])
            parent = min([edit_table[k - 1][l],
                          edit_table[k][l - 1],
                          edit_table[k - 1][l - 1]], key=lambda a: a.comparator_weight)
            if edit_table[k - 1][l] == parent:
                edit_table[k][l].operation = Operation.DELETE
            elif edit_table[k][l - 1] == parent:
                edit_table[k][l].operation = Operation.INSERT
            elif edit_table[k - 1][l - 1] == parent:
                if delta(x[i], y[j]) == 0:
                    edit_table[k][l].operation = Operation.NO_CHANGE
                else:
                    edit_table[k][l].operation = Operation.CHANGE
            edit_table[k][l].weight = parent.comparator_weight
            edit_table[k][l].parent = parent
    q = edit_table[len(x)][len(y)].edit_sequence()
    result = x
    print(result)
    x_i = 0
    y_i = 0
    q.get()
    while not q.empty():
        operation = q.get()
        if operation == Operation.INSERT:
            result = result[:x_i] + y[y_i] + result[x_i:]
            y_i += 1
            x_i += 1
            print(result)
        elif operation == Operation.DELETE:
            result = result[:x_i] + result[x_i + 1:]
            print(result)
        elif operation == operation.CHANGE:
            result = result[:x_i] + y[y_i] + result[x_i + 1:]
            y_i += 1
            x_i += 1
            print(result)
        elif operation == operation.NO_CHANGE:
            y_i += 1
            x_i += 1
    return edit_table[len(x)][len(y)].weight

In [ ]:
edit_distance('los', 'kloc')

In [ ]:
edit_distance('Łódź', 'Lodz')

In [ ]:
edit_distance('kwintesencja', 'quintessence')

In [ ]:
edit_distance('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG')